# PRICE IS RIGHT

In this workshop, we'll design an application that allows a player to guess attributes about a price ie whether it's high, low, or medium as set by the deployer. This is a lot similar to rock paper scissors in that high beats medium which beats low and the winner is decided in that order. Also, the game continues until there is no draw.


We assume this project is created in a directory named

```~/reach/price-is-right```

First, we create the directory which is going to be the home of our project then we change the directory to the newly created folder
```bash
$ mkdir -p ~/reach/workshop-hash-lock && cd ~/reach/price-is-right
```

We assume that after doing the rps tutorial, you have installed reach on your machine. You can confirm this by checking the version with
```bash
../reach version
```

then we start by initializing the reach program with

```bash
../reach init
```

This will create ```index.mjs``` and ```index.rsh``` files in our current directory and populate the files with some stater code as to be seen below.

Problem Analysis
The first step in any program design is to perform problem analysis and determine what information is relevant to the problem. When writing decentralized applications in Reach, this information analysis includes an analysis of the set of participants involved in a computation.
In this case, let's ask the questions:
- Who is involved in this application?
- What information do they know at the start of the program?
- What information will they discover in due course?
- what funds will change and how?


#### Below are proposed answers to the above questions
- This program involves two parties: the payer sending the funds and the receiver of those funds. By tradition, we'll call the first 'Alice' and the second 'Bob'. You might like to use other names, like 'Sender' and 'Receiver'.
- Alice starts off knowing the amount she wants to wager and the guess is high, low, or medium.
- On the other hand, Bob starts with nothing and will mostly depend on Alice's actions. If Alice doesn't deploy Bob can't do a thing about it
- Alice will learn Bob's choice and the result
- Alternatively, Bob will learn the contract, the wager, and finally alice's choice and result
- Alice transfers funds at the beginning and when Bob accepts the wager he also transfers funds to the network. The funds are transferred to the winner after the end of the game.

It's this solution that we are going to try to implement in the course of this workshop.

## Data Definition

The next step of designing our program is representing this information in our program and deciding the participant interact interface for each participant. Which pieces of information go with which participants? Which are functions and which are values?

Let's take a look at our code. In our index.rsh file, we have

```javascript
const Alice = Participant("Alice", {
  ...Deal,
  wager: UInt,
  deadline: UInt,
});

const Bob = Participant("Bob", {
  ...Deal,
  acceptWager: Fun([UInt], Null),
});
```
So here, we set Alice's wager to be a UInt, same with the deadline while for Bob, we have an acceptWager function that takes a UInt(the wager Alice proposed) and returns nothing.

They both inherit the Deal class whose contents are


```javascript
  const Deal = {
    ...hasRandom,
    guessedPrice: Fun([], UInt),
    seeResult: Fun([UInt], Null),
    informTimeout: Fun([], Null),
    informDraw: Fun([], Null),
  };
```
The Deal class takes in the reach's hasRandom function for generating random UInts, has the guessedPrice function which takes in nothing but returns the guessed price. The seeResult takes in a UInt and returns nothing, informTimeout simply informs them of a timeout so takes nothing and returns nothing and pretty much same for informDraw.




## Communication Construction

Now, we can write down the structure of communication and action in our application.



```javascript
// Alice publish her deadline and then pays the wager
Alice.publish(wager, deadline).pay(wager);
commit();

// Bob accepts and pays the wager
Bob.pay(wager).timeout(relativeTime(deadline), () =>
  closeTo(Alice, informTimeout)
);

// While there is no winner
while (result == DRAW) {
  commit();
  // Alice publishes her commit with the deadline stated
  Alice.publish(commitAlice).timeout(relativeTime(deadline), () =>
    closeTo(Bob, informTimeout)
  );
  commit();

  // Bob publishes his guess with the deadline
  Bob.publish(guessBob).timeout(relativeTime(deadline), () =>
    closeTo(Alice, informTimeout)
  );
  commit();

  // Alice publishes her salt and guess so it becomes public after bob has accepted the wager and publish his guess
  Alice.publish(saltAlice, guessAlice).timeout(relativeTime(deadline), () =>
    closeTo(Bob, informTimeout)
  );
}

// We transfer the wager to the winner 
transfer(2 * wager).to(result == ALICE_WIN ? Alice : Bob);
commit();

```



## Assertion Insertion

As usual, we should consider what assertions we can add to our program


```javascript
// Here, we assert the outcome of winner is correct based on Alice and Bob's guesses
assert(winner(HIGH, MEDIUM) == ALICE_WIN);
assert(winner(MEDIUM, HIGH) == BOB_WINS);
assert(winner(HIGH, HIGH) == DRAW);

// We create a loop that will ensure that the value in winner is the ultimate outcome.
forall(UInt, (guessAlice) =>
  forall(UInt, (guessBob) => assert(isResults(winner(guessAlice, guessBob))))
);

// To be explained
forall(UInt, (guessedPrice) =>
  assert(winner(guessedPrice, guessedPrice) == DRAW)
);

// States the invariant that the body of the loop does not change the balance in the contract account and that result is a valid result
invariant(balance() == 2 * wager && isResults(result));

// we assert that Bob can't know Alice's guess, based on what the Reach program does
unknowable(Bob, Alice(_guessAlice, _saltAlice));

// Assert that the published values match the original values
checkCommitment(commitAlice, saltAlice, guessAlice);

// Finally, we assert that the final result is either Bob or Alice wins
assert(result == ALICE_WIN || result == BOB_WINS);

```



## Interaction Introduction

Next, we need to insert the appropriate calls to `interact` to communicate with the frontend.

Lets look at the whole program now:

```javascript
"reach 0.1";

const [isGuessedPrice, HIGH, MEDIUM, LOW] = makeEnum(3);
const [isResults, ALICE_WIN, DRAW, BOB_WINS] = makeEnum(3);

const winner = (guessAlice, guessBob) => (guessAlice + (4 - guessBob)) % 3;

assert(winner(HIGH, MEDIUM) == ALICE_WIN);
assert(winner(MEDIUM, HIGH) == BOB_WINS);
assert(winner(HIGH, HIGH) == DRAW);

// we create a loop that will ensure that the value in winner is the ultimate outcome.
forall(UInt, (guessAlice) =>
  forall(UInt, (guessBob) => assert(isResults(winner(guessAlice, guessBob))))
);

forall(UInt, (guessedPrice) =>
  assert(winner(guessedPrice, guessedPrice) == DRAW)
);

// dual functions performed by all paticipants............
const Deal = {
  ...hasRandom,
  guessedPrice: Fun([], UInt),
  seeResult: Fun([UInt], Null),
  informTimeout: Fun([], Null),
  informDraw: Fun([], Null),
};

export const main = Reach.App(() => {
  const Alice = Participant("Alice", {
    ...Deal,
    wager: UInt,
    deadline: UInt,
  });

  const Bob = Participant("Bob", {
    ...Deal,
    acceptWager: Fun([UInt], Null),
  });
  init();

  // Informing each paticipate of a timeout in progam
  const informTimeout = () => {
    each([Alice, Bob], () => {
      interact.informTimeout(); // First interact
    });
  };

  // Alice publish her wager and deadline.......
  Alice.only(() => {
    const wager = declassify(interact.wager);
    const deadline = declassify(interact.deadline);
  });
  Alice.publish(wager, deadline).pay(wager);
  commit();

  // Bob makes his move in the accepting the wager...........
  Bob.only(() => {
    interact.acceptWager(wager); // Second interact
  });
  Bob.pay(wager).timeout(relativeTime(deadline), () =>
    closeTo(Alice, informTimeout)
  );

  // validating the results to know and check the condition if its a draw then return back to the game run the process again
  var result = DRAW;
  invariant(balance() == 2 * wager && isResults(result));
  while (result == DRAW) {
    commit();
    // The first one to publish deploys the contract
    Alice.only(() => {
      const _guessAlice = interact.guessedPrice(); // Third interact
      const [_commitAlice, _saltAlice] = makeCommitment(interact, _guessAlice);
      const commitAlice = declassify(_commitAlice);
    });
    Alice.publish(commitAlice).timeout(relativeTime(deadline), () =>
      closeTo(Bob, informTimeout)
    );
    commit();

    // Here we hiding the price picked by Alice from Bob before he publishes his guessed price
    unknowable(Bob, Alice(_guessAlice, _saltAlice));
    //  The second one to publish always attaches
    Bob.only(() => {
      const guessBob = declassify(interact.guessedPrice()); // Fourth interact
    });
    Bob.publish(guessBob).timeout(relativeTime(deadline), () =>
      closeTo(Alice, informTimeout)
    );
    commit();

    // here we make Alice publish her guess so it becomes public after bob has accepted the wager and publish his guess
    Alice.only(() => {
      const saltAlice = declassify(_saltAlice);
      const guessAlice = declassify(_guessAlice);
    });

    Alice.publish(saltAlice, guessAlice).timeout(relativeTime(deadline), () =>
      closeTo(Bob, informTimeout)
    );
    checkCommitment(commitAlice, saltAlice, guessAlice);

    result = winner(guessAlice, guessBob);
    continue;
  }

  assert(result == ALICE_WIN || result == BOB_WINS);
  transfer(2 * wager).to(result == ALICE_WIN ? Alice : Bob);
  commit();

  // There each paticpate see the outcome of the steps that take.
  each([Alice, Bob], () => {
    interact.seeResult(result); // Fifth interact
  });
  // exit();
});
```
+ The first interact on line 46 is to inform both participants that there has been a timeout
+ The second interact on line 60 is when bob accepts the wager
+ The third interact on line 73 is for when Alice guesses
+ The fourth interact on line 86 is for when Bob guesses
+ The fifth interact on line 114 is when the game is over and the both participants see the results


## Deployment Decisions

At this point, we need to decide how we're going to deploy this program and really use it in the world. We need to decide how to deploy the contract, as well as what kind of user interaction modality we'll implement inside of our frontend.

Here is the frontend functionality for this DApp (index.mjs)

```javascript
import { loadStdlib } from "@reach-sh/stdlib";
import * as backend from "./build/index.main.mjs";
const stdlib = loadStdlib(process.env);

const startingBalance = stdlib.parseCurrency(100);

const [accAlice, accBob] = await stdlib.newTestAccounts(2, startingBalance);
console.log("Hello, Alice and Bob!");

// formatting the currency to a 4 decemal place...........
const fmt = (x) => stdlib.formatCurrency(x, 4);

const getBalance = async (who) => fmt(await stdlib.balanceOf(who));

// Getting the balace before the game starts for alice and bob
const AliceBeforeBalance = await getBalance(accAlice);
const BobBeforeBalance = await getBalance(accBob);

console.log("Launching...");
const ctcAlice = accAlice.contract(backend);
const ctcBob = accBob.contract(backend, ctcAlice.getInfo());

const GUESSEDPRICE = ["High", "Medium", "Low"];
const RESULT = ["Alice wins", "Draw", "Bob wins"];

const Deal = (Who) => ({
  ...stdlib.hasRandom,
  guessedPrice: async () => {
    const guessedPrice = Math.floor(Math.random() * 3);
    console.log(`${Who} guessed ${GUESSEDPRICE[guessedPrice]}`);
    if (Math.random() <= 0.05) {
      for (let i = 0; i < 10; i++) {
        console.log(`${Who} Takes their time sending back`);
        await stdlib.wait(1);
      }
    }
    return guessedPrice;
  },
  seeResult: (result) => {
    console.log(`${Who} saw the ${RESULT[result]}`);
  },

  informTimeout: () => {
    console.log(`${Who} Observed a timeout`);
  },
  informDraw: () => {
    console.log(`${Who} saw the result it was a draw`);
  },
});

await Promise.all([
  backend.Alice(ctcAlice, {
    // implement Alice's interact object here
    ...Deal("Alice"),
    wager: stdlib.parseCurrency(5),
    deadline: 10,
  }),
  backend.Bob(ctcBob, {
    // implement Bob's interact object here
    ...Deal("Bob"),
    acceptWager: (amt) => {
      console.log(`Bob accepts the ${fmt(amt)}`);
    },
  }),
]);

// getting the balance of bob and alice after wager has been placed.
const afteAclice = await getBalance(accAlice);
const afterBob = await getBalance(accBob);

// remitting funds after the winner has been decalred

console.log(`Alice moved from ${AliceBeforeBalance} to ${afteAclice}`);
console.log(`Bob moved from ${BobBeforeBalance} to ${afterBob}`);
```
With the frontend in place, we can now run `../reach run` and it looks like

```bash
$ ../reach run
Bob asked to give the preimage.
Returning: 40816662354916515903581596667174503941307255426903039386763272451578996162763
Alice went from 100.0 to 74.999999999999823944.
Bob went from 100.0 to 124.999999999999978599.
```




## Discussions and Next Steps

Wohoo!! You made it to the end of this workshop. Hope you enjoyed it and learnt new stuff.

Congratulations once again, and have a good day!